In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [40]:
import matplotlib.pyplot as plt
import numpy as np
import joblib
import random
import pandas as pd

In [ ]:
# levantar datos
features = joblib.load("/content/drive/My Drive/repo_tesis/data/FEATURES_W200_I50.joblib")
label = joblib.load("/content/drive/My Drive/repo_tesis/data/label_W200_I50.joblib")

Esto esta resuelto en mlpAnalysis2, voy a tomar el codigo

Voy a dividir los datos a mano de la siguiente forma. Un sujeto repite cada gesto entre 5 y 6 veces. Voy a usar:

Una repetición para test.
Una para validacion.
Las restantes para train.
En este sentido, voy a guardar los indices de label en listas distintas, para identificar luego que filas de FEATURES tengo que usar para cada conjunto de datos (train, val y test).

La implementación de esta idea parte del supuesto de que los gestos iguales estan agrupados, es decir, las repeticiones del mismo gesto son consecutivas. Esto para los gestos que no son reposo.

Para los gestos de reposo se va a hacer un trato diferenciado. Primero se va a hacer la partición sin ellos, y luego, cada repetición de reposo se va a incluir en el mismo conjunto de la repetición de gesto previa a ese reposo.

VOY A HACERLO DISTINTO.

En lugar de divirlo en 3, lo voy a dividir en 10 partes aproximadamente iguales.
El problema es que cada repeticion de gesto tiene generalmente 5 o 6 ventanas.
Voy a usar la siguiente idea: cada fold será una persona a la que le tengo que repartir cartas. Las repeticiones son las cartas que tengo. Reparto las que tengo, y si no me alcanza para darle una carta a cada persona, uso las repeticiones siguientes para terminar de repartir las cartas.

In [ ]:
label = np.array(label)

# Esto funciona porque los gestos iguales estan agrupados
# Hago un diff en el id del gesto para saber en que indices se cambia el gesto
# Calculo los indices de label que son ventanas de reposo para trabajar con ellas por separado
ind_reposo = np.where(label[:, 1]==0)[0]  # digamos que el reposo es el gesto "0", se identifica en label con ese numero

# Calculo lo contrario ahora para facilitar obtener los labels de las ventanas con gestos
ind_gestos = np.where(label[:, 1]!=0)[0]

label_gestos = label[ind_gestos, :]

division_gestos = np.where(np.diff(label_gestos[:, 1])!=0)[0]

# Ahora hago un diff sobre la id de ventana para saber cuando cambia. Esto me va a servir para poder separarlo.
# ACTUALIZACION: EN REALIDAD SERIA REPETICIONES, EL NOMBRE CONFUNDE. el nombre correcto seria division_repeticiones o algo asi

# en train, val y test
division_id_ventanas = np.where(np.diff(label_gestos[:, 0])!=0)[0]

cant_folds = 10
# quiero una lista con 10 listas si trabajo con 10 folds
# Crear una lista vacía
folds_repeticiones = []

# Iterar 10 veces para crear 10 listas
for _ in range(cant_folds):
    lista_interna = []  # Crear una lista interna vacía
    folds_repeticiones.append(lista_interna)  # Agregar la lista interna a la lista principal

# hago la division de gestos
ind_jugador = random.randint(0, cant_folds-1)   #  jugador (fold) que le toca recibir la primera carta (repeticion)
ronda_terminada = False  # indica si me paso de rango, es decir que termino la ronda y tengo que empezar de nuevo

for i in range(len(division_gestos)+1):  #
    if i==len(division_gestos):
        ind_tope_ventana = np.max(label_gestos[:, 0])
        ind_inicio_ventana = label_gestos[division_gestos[-1], 0]

        # cantidad de repeticiones en el gesto actual (del sujeto actual)
        cant_ventanas = ind_tope_ventana - ind_inicio_ventana

    else:
        tope = division_gestos[i]   # este es limite con el proximo gesto

        if i==0:
            inicio = 0
            ind_inicio_ventana = 0
        else:
            inicio = division_gestos[i-1]
            ind_inicio_ventana = np.where(division_id_ventanas==inicio)[0][0]

        # busco en division_id_ventana cuantos elementos hay entre inicio y tope
        ind_tope_ventana = np.where(division_id_ventanas==tope)[0][0]

        # cantidad de repeticiones en el gesto actual (del sujeto actual)
        cant_ventanas = ind_tope_ventana - ind_inicio_ventana

    if i != 0:
        # sorteo las ventanas que van a cada grupo
        sorteo = np.random.permutation(cant_ventanas)
        sorteo += 1     # correccion de rango
    else:
        sorteo = np.random.permutation(cant_ventanas+1)

    # reparto las cartas
    ronda_terminada = False
    for k in range(len(sorteo)):
      ind_jugador += 1    # para repartirle carta al siguiente jugador
      if ind_jugador > (cant_folds-1):   # si termino la ronda tengo que comienzar una nueva
        ronda_terminada = True
        ind_real = ind_jugador - cant_folds   # al restar 10 vuelvo al principio de la ronda
        #print(ind_real, ind_jugador)
        folds_repeticiones[ind_real].append(ind_inicio_ventana + sorteo[k])
      else:
        #print(ind_jugador)
        folds_repeticiones[ind_jugador].append(ind_inicio_ventana + sorteo[k])

    if ronda_terminada:
      ind_jugador = ind_jugador - cant_folds  # para volver al rango de la ronda

A continuación se inserta un código de visualización. Fue usado para chequear que se estaba trabajando correctamente. Quizás pueda ser de utilidad en algún momento.

In [ ]:
# Quiero visualizar con diferentes colores las ventanas que van al train, val y test
ventanas_train = []
ventanas_val = []
ventanas_test = []

# plt.figure()
# plt.plot(label_gestos[:, 1]/np.max(label_gestos[:, 1]), label = 'gesto')
# plt.plot(label_gestos[:, 0]/np.max(label_gestos[:, 0]), label = 'ventana')
# for i in range(len(train)): #
#     ventana_act_train = np.where(label_gestos[:, 0]==train[i])[0]
#     ventanas_train.append(ventana_act_train)
#     plt.plot(ventana_act_train, label_gestos[ventana_act_train, 0]/np.max(label_gestos[:, 0]), color = 'green')

#     if i < len(val):   # val y test tienen menos datos
#         ventana_act_val = np.where(label_gestos[:, 0]==val[i])[0]
#         ventanas_val.append(ventana_act_val)
#         plt.plot(ventana_act_val, label_gestos[ventana_act_val, 0]/np.max(label_gestos[:, 0]), color = 'blue')

#         ventana_act_test = np.where(label_gestos[:, 0]==test[i])[0]
#         ventanas_test.append(ventana_act_test)
#         plt.plot(ventana_act_test, label_gestos[ventana_act_test, 0]/np.max(label_gestos[:, 0]), color = 'black')

# plt.legend()

Procedo a armar los conjuntos de train, val y test para las ventanas con gesto. Tengo identificada que repeticion va para cada conjunto, ahora identifico los indices de las features usando label.

In [27]:
# Procedo a armar los conjuntos de train, val y test para las ventanas con gesto
# Tengo identificada que repeticion va para cada conjunto, ahora identifico los indices de las features usando label

folds_ind = []
# Iterar 10 veces para crear 10 listas, esto es porque tengo 10 folds
for _ in range(cant_folds):
    lista_interna = []  # Crear una lista interna vacía
    folds_ind.append(lista_interna)  # Agregar la lista interna a la lista principal

# Ahora con la estructura de 10 listas creada, recorro cada una de ellas
for n in range(len(folds_ind)):
  for j in range(len(folds_repeticiones[n])): # para cada fold, me fijo que indices de datos comprenden las repeticiones que reparti antes
    folds_ind[n].extend(np.where(label[:, 0]==folds_repeticiones[n][j])[0])


In [34]:
# Ahora agrego las ventanas de reposo

# Voy a fijarme a que conjunto fue la repeticion anterior (que es de un gesto que no es reposo), y mandar la repeticion de reposo para ese conjunto.

# Al igual que para los otros gestos, quiero mandar todas las features de una misma "repeticion" de reposo para el mismo conjunto
div_repe_reposo = np.where(np.diff(ind_reposo)!=1)[0]
div_repe_reposo = np.insert(div_repe_reposo, 0, -1)   # el -1 es porque todos me apuntan al ultimo elemento de cada repeticion, en el for le voy a sumar 1 a todos para que me apunten al inicio que es lo que me interesa
# si quedan dudas del comentario anterior, imprimir pd.DataFrame(ind_reposo)

In [47]:
pd.DataFrame(label)

,0,1,2
0,0,1,1
1,0,1,1
2,0,1,1
3,0,1,1
4,0,1,1
...,...,...,...
64521,1419,0,10
64522,1419,0,10
64523,1419,0,10
64524,1419,0,10


In [35]:
print(div_repe_reposo)

[  -1    5   11   17   23   29   35   41   47   52   58   64   70   76
   82   88   94  100  106  111  117  123  129  135  141  147  153  160
  166  172  178  184  190  196  202  208  215  221  227  233  239  245
  251  257  263  269  275  282  288  294  300  305  311  317  323  330
  336  341  347  353  359  365  371  377  383  389  396  402  408  414
  420  426  433  440  446  453  460  467  473  480  487  494  501  508
  515  522  529  536  543  550  557  564  571  578  585  592  599  606
  614  621  628  635  643  650  657  664  671  679  686  693  700  707
  715  722  729  736  743  751  758  765  772  779  786  793  800  807
  815  822  830  838  845  853  860  867  875  882  889  896  903  910
  917  924  931  937  943  949  955  961  967  973  979  985  991  997
 1003 1009 1014 1020 1026 1032 1038 1044 1050 1056 1062 1067 1073 1079
 1085 1092 1098 1104 1110 1116 1122 1128 1134 1140 1146 1153 1159 1165
 1171 1177 1183 1189 1195 1201 1207 1214 1220 1226 1231 1237 1243 1249
 1255 

In [42]:
pd.DataFrame(ind_reposo)

,0
0,75
1,76
2,77
3,78
4,79
...,...
4958,64521
4959,64522
4960,64523
4961,64524


In [44]:
# Calculo cantidad de repeticiones de reposo
cant_repeticiones_reposo = len(div_repe_reposo)

pertenencia_conjuntos = []  # train, val, test. Cada dato solo puede pertenecer a 1 de estos conjuntos

for i in range(len(div_repe_reposo)):
    ind0_repe_reposo_act = ind_reposo[div_repe_reposo[i]+1]  # porque antes tenia el indice del dato final de la repe anterior
    id_repe_act = label[ind0_repe_reposo_act, 0]  # me fijo en label el id de la repe actual
    id_repe_ant = label[ind0_repe_reposo_act-1, 0]  # me fijo en label el id de la repe anterior, donde voy a guardar la repe actual

    # Iterar sobre las sub-listas dentro de la lista principal
    for i, sublista in enumerate(folds_repeticiones):
        # Verificar si el ID está en la sub-lista actual
        if id_repe_ant in sublista:
            indice_sublista = i
            break  # Terminar el bucle si se encuentra el ID

    # Verificar si se encontró el ID y en qué sub-lista está
    if indice_sublista is not None:
        print(f"El ID {id_repe_ant} se encuentra en la sub-lista {indice_sublista} de la lista principal.")
    else:
        print("El ID no se encuentra en ninguna de las sub-listas.")

    # id de repeticio de reposo y a que conjunto mandarlo
    pertenencia_conjuntos.append([id_repe_act, indice_sublista])

pertenencia_conjuntos = np.array(pertenencia_conjuntos)

El ID 0 se encuentra en la sub-lista 1 de la lista principal.
El ID 1 se encuentra en la sub-lista 9 de la lista principal.
El ID 2 se encuentra en la sub-lista 0 de la lista principal.
El ID 3 se encuentra en la sub-lista 2 de la lista principal.
El ID 4 se encuentra en la sub-lista 3 de la lista principal.
El ID 5 se encuentra en la sub-lista 4 de la lista principal.
El ID 6 se encuentra en la sub-lista 5 de la lista principal.
El ID 7 se encuentra en la sub-lista 6 de la lista principal.
El ID 8 se encuentra en la sub-lista 7 de la lista principal.
El ID 9 se encuentra en la sub-lista 8 de la lista principal.
El ID 10 se encuentra en la sub-lista 0 de la lista principal.
El ID 11 se encuentra en la sub-lista 9 de la lista principal.
El ID 12 se encuentra en la sub-lista 2 de la lista principal.
El ID 13 se encuentra en la sub-lista 6 de la lista principal.
El ID 14 se encuentra en la sub-lista 1 de la lista principal.
El ID 15 se encuentra en la sub-lista 3 de la lista principal.
El

In [54]:
for i in range(len(pertenencia_conjuntos)):
  ind_fold = np.where(label[:, 0]==pertenencia_conjuntos[i, 0])[0]
  numero_fold = pertenencia_conjuntos[i, 1]
  # agrego los indices de reposo al fold correspondiente segun pertenencia_conjuntos
  folds_ind[numero_fold].extend(ind_fold)

In [46]:
pd.DataFrame(pertenencia_conjuntos)

,0,1
0,710,1
1,711,9
2,712,0
3,713,2
4,714,3
...,...,...
705,1415,8
706,1416,4
707,1417,7
708,1418,5


In [63]:
pd.DataFrame(folds_ind[9])

,0
0,81
1,82
2,83
3,84
4,85
...,...
6403,63957
6404,63958
6405,63959
6406,63960


In [64]:
joblib.dump(folds_ind, '10foldsXRepeticion_W200_I50.pkl')

['10foldsXRepeticion_W200_I50.pkl']